In [1]:
### Beginning by loading libraries and project data

In [2]:
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


<mark> Will need to find a way to load in the data that does not require the use of additional files

In [3]:
player_data <- read_csv("players.csv") |> select(-hashedEmail)
player_data

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,played_hours,name,gender,Age
<chr>,<lgl>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,30.3,Morgan,Male,9
Veteran,TRUE,3.8,Christian,Male,17
Veteran,FALSE,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,0.0,Dylan,Prefer not to say,17
Amateur,FALSE,2.3,Harlow,Male,17
Pro,TRUE,0.2,Ahmed,Other,NA


In [4]:
sessions_data <- read_csv("sessions.csv") |> select(-hashedEmail)
sessions_data

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<dbl>,<dbl>
30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
⋮,⋮,⋮,⋮
28/07/2024 15:36,28/07/2024 15:57,1.72218e+12,1.72218e+12
25/07/2024 06:15,25/07/2024 06:22,1.72189e+12,1.72189e+12
20/05/2024 02:26,20/05/2024 02:45,1.71617e+12,1.71617e+12


It seems that the easiest question to answer will be quesiton 3: We are interested in demand forecasting, namely, what time windows are most likely to have large number of simultaneous players. This is because we need to ensure that the number of licenses on hand is sufficiently large to accommodate all parallel players with high probability. 

My rough plan is as follows: load in the sessions data. Then, group the data by time of day that it is played. Plot the data by time of day. Then I can also look at what days of the week are most popular. Once I have looked at the data, I can decide if I should look at the month as well.

In [19]:
### Tidying the data


sessions_tidy <- sessions_data |>
                    select(start_time, end_time) |>
                    separate(col = start_time, into = c("date_start", "time_start"), sep = " ") |>
                    separate(col = end_time, into = c("date_end", "time_end"), sep = " ") |>
                    separate(col = date_start, into = c("day_start", "month_start", "year_start"), sep = "/") |>
                    separate(col = date_end, into = c("day_end", "month_end", "year_end"), sep = "/") |>
                    separate(col = time_start, into = c("hour_start", "minute_start"), sep = ":") |>
                    separate(col = time_end, into = c("hour_end", "minute_end"), sep = ":") 




sessions_tidy

day_start,month_start,year_start,hour_start,minute_start,day_end,month_end,year_end,hour_end,minute_end
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
30,06,2024,18,12,30,06,2024,18,24
17,06,2024,23,33,17,06,2024,23,46
25,07,2024,17,34,25,07,2024,17,57
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
28,07,2024,15,36,28,07,2024,15,57
25,07,2024,06,15,25,07,2024,06,22
20,05,2024,02,26,20,05,2024,02,45
